<img src="http://prosilico.com/assets/static/media/prosilico_logo_2x.png" width="400"/> <img src="https://openrisknet.org/static/images/orn-logo-lg.png" width="400" align="right">



# Accessing ProSilico services in OpenRiskNet via Jupyter Notebook

Ola Spjuth, 2019-10-23

## Overview

[ProSilico](http://prosilico.com/) has developed the product [ProSilico ADME/PK Studio](http://prosilico.com/products) as a Web-based GUI for their customers. This application delivers estimates of the following human ADME/PK-parameters:

### Primary human ADME/PK-parameters

|Endpoint | Abbreviation | Description
:--- | :--- | :--- 
Fraction absorbed | fabs | Predicts passive intestinal permeability-based fraction absorbed in-vivo in man (not considering solubility, active transport or instability in GI fluids)
Intrinsic hepatic metabolic clearance | CLint | Predicts intrinsic hepatic metabolic clearance in-vivo in man (phase I metabolism and conjugation)
Steady-state volume of distribution | Vss | Predicts steady-state volume of distribution in-vivo in man (not considering enterohepatic circulation)
Fraction unbound in human plasma | fu | Predicts in-vitro fraction unbound in human plasma.
Maximum in-vivo solubility/dissolution potential | fdiss| Predicts the maximum solubility/dissolution potential in the human gastrointestinal tract in-vivo following oral dosing. A value of 40 % shows that maximally 40 % of an oral dose is anticipated to be dissolved using an optimal oral formulation. A value of 100 % indicates complete in-vivo solubility/dissolution potential with an optimized oral formulation.


### Secondary human ADME/PK-parameters

Endpoint | Abbreviation | Description
--- | --- | --- 
Hepatic clearance | CLH | Calculated based on predicted CLint and fu, hepatic blood flow (1500 mL/min) and the well-stirred liver model.
Oral bioavailability | F | Calculated based on predicted fabs (not considering active transport or gut-wall metabolism), fdiss and CLH.
Half-life | t2 | Calculated based on predicted CLH and Vss (not considering excretion or enterohepatic circulation).

## Modeling details
ProSilico models make use of Conformal Prediction methodology that outputs valid prediction intervals at user-specified levels of confidence. The validity of conformal prediction is theoretically proven, see [vovk et al, 2005](http://alrw.net).
For an introduction to conformal prediction in cheminformatics, we refer to [Norinder et al., J Chem Inf Model 54:6 2014](https://pubs.acs.org/doi/10.1021/ci5001168).


## Implementation in OpenRiskNet
ProSilico models have been integrated and is running on the OpenRiskNet infrastructure. In the notebook below, I showcase how easy it is to call the service.

#### Remaining tasks
* Implement authorization using KeyCloak
* Manage ProSilico licenses in a good manner


In [154]:
import requests
from IPython.display import Image
from IPython.core.display import HTML
import json
import pprint

prosilicobaseURL='http://prosilico.prod.openrisknet.org/prosilico?molecule='

Let us now predict a SMILES at a specific confidence level:

In [155]:
smiles='CC1=CN=C(C(=C1OC)C)CS(=O)C2=NC3=C(N2)C=C(C=C3)OC'
confidence = 0.7

Structure the REST call and parse the JSON results:

In [156]:
prosilicoCall = prosilicobaseURL + smiles + '&confidence=' + str(confidence)
prosilicoResultJSON = requests.get(prosilicoCall).content
prosilicoResult = json.loads(prosilicoResultJSON)

Let us inspect the full JSON result:

In [157]:
pprint.pprint(prosilicoResult)

{'CLH': 505.5425602239618,
 'CLint': {'lower': 2890.6798823654744,
           'mid': 19054.607179632483,
           'upper': 74131.02413009177,
           'url': 'http://m37-modelingweb-deploy.osprod.pharmb.io/v1/predict?&molecule=CC1%3DCN%3DC(C(%3DC1OC)C)CS(%3DO)C2%3DNC3%3DC(N2)C%3DC(C%3DC3)OC&confidence=0.7'},
 'F': 0.49517350784581526,
 'Vss': {'lower': 0.11117317272815914,
         'mid': 0.23988329190194904,
         'upper': 0.5176068319505677,
         'url': 'http://m38-modelingweb-deploy.osprod.pharmb.io/v1/predict?&molecule=CC1%3DCN%3DC(C(%3DC1OC)C)CS(%3DO)C2%3DNC3%3DC(N2)C%3DC(C%3DC3)OC&confidence=0.7'},
 'fabs': {'lower': 0.834,
          'mid': 0.97,
          'upper': 1,
          'url': 'http://m39-modelingweb-deploy.osprod.pharmb.io/v1/predict?&molecule=CC1%3DCN%3DC(C(%3DC1OC)C)CS(%3DO)C2%3DNC3%3DC(N2)C%3DC(C%3DC3)OC&confidence=0.7'},
 'fdiss': {'lower': 0.438,
           'mid': 0.77,
           'upper': 1,
           'url': 'http://m40-modelingweb-deploy.osprod.pharmb.

We can extract the five primary endpoints:

In [158]:
pprint.pprint(prosilicoResult['rawData'])

{'CLintRaw': {'lower': 3.461, 'mid': 4.28, 'upper': 4.87},
 'VssRaw': {'lower': -0.954, 'mid': -0.62, 'upper': -0.286},
 'fabsRaw': {'lower': 0.834, 'mid': 0.97, 'upper': 1},
 'fdissRaw': {'lower': 0.438, 'mid': 0.77, 'upper': 1.103},
 'fuRaw': {'lower': 0.718, 'mid': 1.38, 'upper': 2.042}}


We can also look at the there secondary endpoints:

In [159]:
print('CLH:\t'+str(prosilicoResult['CLH']))
print('F:\t'+str(prosilicoResult['F']))
print('t2:\t'+str(prosilicoResult['t2']))

CLH:	505.5425602239618
F:	0.49517350784581526
t2:	0.41092273357767717


## Integration with other OpenRiskNet services

We will do a simple demonstration of calling OpenRiskNet services

In [160]:
callUrl = 'https://cplogd.prod.openrisknet.org/v1/prediction?smiles='+smiles + '&confidence=' + str(confidence)
imageURL='https://cplogd.prod.openrisknet.org/v1/predictionImage?smiles='+smiles +'&imageWidth=600&imageHeight=600&addTitle=false&confidence=' + str(confidence)
result=requests.get(callUrl).content
print(result)
Image(url= imageURL)

b'{"predictionMidpoint":2.38,"smiles":"O=S(C1=NC=2C=CC(OC)=CC2N1)CC3=NC=C(C(OC)=C3C)C","lower":2.102,"upper":2.658,"confidence":0.7}'
